In [5]:
import pandas as pd

papers = pd.read_json("../src/paper_data.json")
paper_data = papers["data"]
paper_data = paper_data.tolist()
paper_data.append("/2401.02412.json")
paper_data.append("/2503.10622.json")

In [6]:
import json

all_discussions = {}
lengths = []

for paper in paper_data:
    with open(f".{paper}", "r") as f:
        data = json.load(f)
        ind_counter = 0
        all_counter = 0
        discussions = []
        for post in data["posts"].values():
            if "tweet_type" not in post or "is_branch" not in post:
                continue
            if "in_reply_to_status_id_str" not in post and "quoted_status_id_str" not in post:
                ind_counter += 1
            all_counter += 1
            if not post["is_branch"]:
                continue
            if post["tweet_type"] in ["Q&A", "Critique", "Perspective"]:
                discussions.append(post)
                for reply in post["replies"]:
                    if reply in data["posts"]:
                        discussions.append(data["posts"][reply])
        lengths.append((ind_counter, all_counter, len(data["posts"])))
        discussions = [
            {"id": p["id_str"], "text": " ".join([e["text"] for e in p["entities"] if e["type"] == "text"]), "reply_to": p.get("in_reply_to_status_id_str", None), "quoted": p.get("quoted_status_id_str", None)} for p in discussions
        ]
        all_discussions[paper.split(".json")[0].split("/")[1]] = discussions
lengths

[(17, 77, 838),
 (17, 67, 940),
 (15, 86, 507),
 (4, 46, 279),
 (25, 80, 983),
 (18, 78, 434),
 (14, 62, 268),
 (16, 104, 845),
 (8, 77, 1099),
 (17, 86, 1254),
 (15, 78, 1041),
 (44, 105, 1685),
 (12, 33, 585),
 (10, 100, 578)]

In [7]:
for key, value in all_discussions.items():
    with open(f"./{key}_discussion.json", "w") as f:
        json.dump(value, f, indent=None)